# Re_entreno

Añadimos acceso a la carpeta de los dataset del Drive del grupo de proyectos

Importante estar ya logeados en la cuenta del grupo, da problemas si estamos en nuestra cuenta personal y queremos montar el drive de la cuenta del grupo.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importamos las bibliotecas necesarias

In [3]:
import torch
torch.cuda.is_available()

False

In [4]:
# Para windows con la CPU:
!pip3 install torch torchvision torchaudio 

# Para Windows con CUDA:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

# Para windows con la CPU:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# Para Windows con CUDA:
# !pip3 install torch torchvision torchaudio

# Es posible que la mayoría de dependencias ya esten instaladas ^^

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!git clone https://github.com/ultralytics/yolov5
# Se deberia haber creado una carpeta en nuestro area de trabajo
# Si ya lo teniamos saldra el siguiente error =>fatal: destination path 'yolov5' already exists and is not an empty directory.

Cloning into 'yolov5'...
remote: Enumerating objects: 15532, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 15532 (delta 51), reused 118 (delta 43), pack-reused 15393
Receiving objects: 100% (15532/15532), 14.59 MiB | 22.33 MiB/s, done.
Resolving deltas: 100% (10575/10575), done.


In [6]:
!cd yolov5 & pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [24]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2 #Para abrir webcam
# cv2.imshow() is disabled in Colab, because it causes Jupyter sessionsto crash, consider using cv2_imshow 
from google.colab.patches import cv2_imshow 
import sys # to access the system

# Esta línea de código establece una variable de entorno llamada KMP_DUPLICATE_LIB_OK con el valor "True", lo que significa que permite la carga de bibliotecas duplicadas en el entorno de Python. Esto es útil para algunas bibliotecas de Machine Learning que usan OpenMP para acelerar el cómputo.
import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

***
En la siguiente celda se escogen 100 imagenes para hacer el re-entreno.
 
Además esas imagenes se suben a unas carpetas que creamos en el directorio base de colabs.
***

In [ ]:
img = cv2.imread('/content/drive/MyDrive/IA/Datasets/YoloV5/00001.jpg', cv2.IMREAD_UNCHANGED)
cv2_imshow(img)

In [35]:
import os
import random
import shutil

# Creamos los directorios necesarios
os.makedirs('/content/data/images', exist_ok=True)
os.makedirs('/content/data/labels', exist_ok=True)

random.seed(15)

src = "/content/drive/MyDrive/IA/Datasets/YoloV5/"
dest_Image = "/content/data/images/"
dest_Label = "/content/data/labels/"

lista_aleatorios_rellenada = []

for n in range(100):
  aleatorio = random.randint(0,900)
  aleatorio_rellenado = str(aleatorio).zfill(5)
  src_img = src + str(aleatorio_rellenado) + ".jpg"
  if os.path.exists(src_img):
    if aleatorio_rellenado not in lista_aleatorios_rellenada:
      lista_aleatorios_rellenada.append(aleatorio_rellenado)

# print(lista_aleatorios_rellenada)

for num in lista_aleatorios_rellenada:
  src_img = src + str(num) + ".jpg"
  dest_img = dest_Image + str(num) + ".jpg"
  shutil.copy2(src_img, dest_img)
  # print(dest_img)

for num in lista_aleatorios_rellenada:
  src_lab = src + str(num) + ".txt"
  dest_lab = dest_Label + str(num) + ".txt"
  shutil.copy2(src_lab, dest_lab)
  # print(dest_lab)

***
**`Entrenar un modelo desde cero`**
***

Ahora tenemos que crear el archivo dataset.yaml, que contiene las categorias de nuestros pesos y las rutas de nuestras imagenes de entrenamiento, test y validación.

In [38]:
f= open("yolov5/data/cien_images_señales.yaml","w+")

f.write("path: ../data \ntrain: images \nval: images \n\nnc: 4 \n\nnames: ['prohibitory', 'danger', 'mandatory', 'other']")

f.close()

In [ ]:
!cd yolov5 && python train.py --img 640 --batch 16 --epochs 500 --data cien_images_señales.yaml --weights yolov5s.pt  --workers 2

requirements: YOLOv5 requirement "gitpython" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00

requirements: 1 package updated per ['gitpython']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

train: weights=yolov5s.pt, cfg=, data=cien_images_señales.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, enti

In [ ]:
# LOGGER.info(( '%10s ' * 7) % ('Epoch', 'gpu_mem', 'box', 'obJ', 'cls', 'labels', 'img_size'))
# if RANK in [-1, 0]:
#     pbar = tqdm(pbar, total=nb, bar_format='{1_bar}{bar:10}{r_bar}{bar:-10b}') #progress_bar

***
**`Cargamos nuestro modelo`**
***

In [ ]:
modelo_100_señales = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp10/weights/best.pt', force_reload=True)